# Introduction to midodrine examination

In [1]:
from google.colab import auth
auth.authenticate_user()
from google.cloud import bigquery

# Initialize client with project ID
client = bigquery.Client(project='sccm-datathon-2025-participant')

In [6]:
query = f"""

WITH midodrine_meds AS (
  SELECT DISTINCT
    m.patientunitstayid,
    m.drugname,
    m.drugstartoffset,
    m.drugstopoffset,
    m.dosage,
    m.routeadmin
  FROM `sccm-discovery.eicu_crd_ii_v0_2_0.medication` m
  WHERE (LOWER(m.drugname) LIKE '%midodrine%'
         OR LOWER(m.drugname) LIKE '%proamatine%'
         OR LOWER(m.drugname) LIKE '%orvaten%')
      AND m.drugstartoffset IS NOT NULL
      AND LOWER(COALESCE(m.routeadmin, '')) IN ('po', 'oral', 'enteral', 'gt', 'ng', 'peg', '')
)
SELECT * FROM midodrine_meds;

"""
midodrine_admin = client.query(query).to_dataframe()
midodrine_admin

,patientunitstayid,drugname,drugstartoffset,drugstopoffset,dosage,routeadmin
0,11383203,midodrine hcl 5 mg po tabs,-1075,<NA>,10,PO
1,11728644,MIDODRINE HCL 5 MG PO TABS,2155,2144,5 mg,PO
2,11769218,MIDODRINE HCL 5 MG PO TABS,1155,1181,5 mg,PO
3,12473070,MIDODRINE HCL 5 MG PO TABS,942,5618,5 mg,PO
4,11653357,MIDODRINE HCL 5 MG PO TABS,-23308,-20837,5 mg,Oral
...,...,...,...,...,...,...
2684,12460032,MIDODRINE HCL 5 MG PO TABS,5723,26034,2.5 mg,PO
2685,12466391,MIDODRINE HCL 5 MG PO TABS,-302,-66,5 mg,PO
2686,12507912,MIDODRINE HCL 5 MG PO TABS,10773,14082,5 mg,Oral
2687,11317615,MIDODRINE HCL 5 MG PO TABS,-434,-429,10 mg,PO


In [8]:
query = f"""

WITH medication_vaso AS (
    SELECT DISTINCT
        m.patientunitstayid,
        m.drugname,
        m.drugstartoffset,
        m.drugstopoffset,
        CASE
            WHEN LOWER(m.drugname) LIKE '%norepinephrine%' OR LOWER(m.drugname) LIKE '%noradrenaline%' OR LOWER(m.drugname) LIKE '%levophed%' THEN 'norepinephrine'
            WHEN LOWER(m.drugname) LIKE '%epinephrine%' OR LOWER(m.drugname) LIKE '%adrenaline%' THEN 'epinephrine'
            WHEN LOWER(m.drugname) LIKE '%dopamine%' THEN 'dopamine'
            WHEN LOWER(m.drugname) LIKE '%vasopressin%' OR LOWER(m.drugname) LIKE '%pitressin%' THEN 'vasopressin'
            WHEN LOWER(m.drugname) LIKE '%phenylephrine%' OR LOWER(m.drugname) LIKE '%neosynephrine%' THEN 'phenylephrine'
            WHEN LOWER(m.drugname) LIKE '%dobutamine%' THEN 'dobutamine'
        END as vasopressor_type
    FROM `sccm-discovery.eicu_crd_ii_v0_2_0.medication` m
    WHERE (LOWER(m.drugname) LIKE '%norepinephrine%' OR LOWER(m.drugname) LIKE '%noradrenaline%' OR LOWER(m.drugname) LIKE '%levophed%'
           OR LOWER(m.drugname) LIKE '%epinephrine%' OR LOWER(m.drugname) LIKE '%adrenaline%'
           OR LOWER(m.drugname) LIKE '%dopamine%' OR LOWER(m.drugname) LIKE '%intropin%'
           OR LOWER(m.drugname) LIKE '%vasopressin%' OR LOWER(m.drugname) LIKE '%pitressin%'
           OR LOWER(m.drugname) LIKE '%phenylephrine%' OR LOWER(m.drugname) LIKE '%neosynephrine%'
           OR LOWER(m.drugname) LIKE '%dobutamine%' OR LOWER(m.drugname) LIKE '%dobutrex%')
        AND m.drugstartoffset IS NOT NULL
),
infusion_vaso AS (
    SELECT DISTINCT
        i.patientunitstayid,
        i.drugname,
        i.infusionoffset as drugstartoffset,
        LEAD(i.infusionoffset) OVER (PARTITION BY i.patientunitstayid, i.drugname ORDER BY i.infusionoffset) as drugstopoffset,
        CASE
            WHEN LOWER(i.drugname) LIKE '%norepinephrine%' OR LOWER(i.drugname) LIKE '%noradrenaline%' THEN 'norepinephrine'
            WHEN LOWER(i.drugname) LIKE '%epinephrine%' OR LOWER(i.drugname) LIKE '%adrenaline%' THEN 'epinephrine'
            WHEN LOWER(i.drugname) LIKE '%dopamine%' THEN 'dopamine'
            WHEN LOWER(i.drugname) LIKE '%vasopressin%' THEN 'vasopressin'
            WHEN LOWER(i.drugname) LIKE '%phenylephrine%' THEN 'phenylephrine'
            WHEN LOWER(i.drugname) LIKE '%dobutamine%' THEN 'dobutamine'
        END as vasopressor_type
    FROM `sccm-discovery.eicu_crd_ii_v0_2_0.infusiondrug` i
    WHERE (LOWER(i.drugname) LIKE '%norepinephrine%' OR LOWER(i.drugname) LIKE '%noradrenaline%'
           OR LOWER(i.drugname) LIKE '%epinephrine%' OR LOWER(i.drugname) LIKE '%adrenaline%'
           OR LOWER(i.drugname) LIKE '%dopamine%' OR LOWER(i.drugname) LIKE '%vasopressin%'
           OR LOWER(i.drugname) LIKE '%phenylephrine%' OR LOWER(i.drugname) LIKE '%dobutamine%')
        AND SAFE_CAST(i.drugrate AS FLOAT64) > 0
        AND i.infusionoffset IS NOT NULL
)
SELECT * FROM medication_vaso
UNION ALL
SELECT * FROM infusion_vaso;


"""
vasopressin_admin = client.query(query).to_dataframe()
vasopressin_admin

,patientunitstayid,drugname,drugstartoffset,drugstopoffset,vasopressor_type
0,11540556,norEPINEPHrine,886,886,norepinephrine
1,11542563,DOPamine,786,786,dopamine
2,11541451,norepinephrine,-16,-16,norepinephrine
3,12448965,PHENYLEPHRINE HCL,-26,-20,phenylephrine
4,12074693,norepinephrine,-30,-30,norepinephrine
...,...,...,...,...,...
2830396,12157224,phenylephrine,-306,-306,phenylephrine
2830397,11555150,phenylephrine,567,567,phenylephrine
2830398,11741575,phenylephrine,-5750,-5750,phenylephrine
2830399,11557467,phenylephrine,5405,5405,phenylephrine


In [9]:
query = f"""

  SELECT distinct
    p.patientunitstayid,
    p.patienthealthsystemstayid,
    p.hospitalid,
    p.age,
    p.gender,
    p.ethnicity,
    p.hospitaldischargeoffset,
    p.unitdischargeoffset,
    p.unittype,
    CASE
      WHEN p.age = '> 89' THEN 90
      WHEN SAFE_CAST(p.age AS INT64) IS NOT NULL THEN SAFE_CAST(p.age AS INT64)
      ELSE NULL
    END as age_numeric
  FROM `sccm-discovery.eicu_crd_ii_v0_2_0.patient` p
  WHERE p.age != '' AND p.age IS NOT NULL


"""
patient_info = client.query(query).to_dataframe()
patient_info

,patientunitstayid,patienthealthsystemstayid,hospitalid,age,gender,ethnicity,hospitaldischargeoffset,unitdischargeoffset,unittype,age_numeric
0,11733590,9934563,2000536,25,Male,Asian,25375,25153,Neuro ICU,25
1,11765469,9965107,2000536,25,Female,Caucasian,8747,1641,Neuro ICU,25
2,11765468,9965107,2000536,25,Female,Caucasian,31969,23222,Neuro ICU,25
3,11524173,9736821,2000533,21,Male,Caucasian,3166,3166,Med-Surg ICU,21
4,11328348,9554994,2000585,28,Female,Caucasian,5146,5146,Med-Surg ICU,28
...,...,...,...,...,...,...,...,...,...,...
201703,12167911,10268500,2000671,> 89,Male,Other/Unknown,7264,4630,MICU,90
201704,12165886,10266803,2000671,> 89,Male,Hispanic,4740,2916,MICU,90
201705,12366524,10444360,2001097,> 89,Female,Caucasian,4042,2242,Med-Surg ICU,90
201706,12364986,10443011,2001097,> 89,Female,Caucasian,5450,5450,Med-Surg ICU,90


In [ ]:
query = f"""


"""
client.query(query).to_dataframe()

In [ ]:
query = f"""


"""
client.query(query).to_dataframe()

In [ ]:
query = f"""


"""
client.query(query).to_dataframe()